In [2]:
import pandas as pd
import os
import datetime
import pickle
import re
import time

In [3]:
# Regulatory sections with matched noun chunks
df_regSentsExpand=pd.read_pickle('/home/ec2-user/SageMaker/New Uncertainty/Reg Relevance/RegSentsExpand_NounChunks3.pkl')
print(df_regSentsExpand.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 822737 entries, 0 to 822736
Data columns (total 6 columns):
 #   Column                       Non-Null Count   Dtype 
---  ------                       --------------   ----- 
 0   ID                           822737 non-null  object
 1   RegSentsExpand               822737 non-null  object
 2   NounChunksMatch              822737 non-null  int64 
 3   NounChunkMatchWords          822737 non-null  object
 4   NounChunkMatchWordsFiltered  822737 non-null  object
 5   NounChunkMatchFiltered       822737 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 37.7+ MB
None


In [4]:
# All news article metadata
allNews=pd.read_pickle('/home/ec2-user/SageMaker/New Uncertainty/parsed_xml.pkl')
print(allNews.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 822737 entries, 0 to 822736
Data columns (total 12 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   ID             822737 non-null  object        
 1   Title          822737 non-null  object        
 2   Type           822737 non-null  object        
 3   StartDate      822737 non-null  datetime64[ns]
 4   EndDate        822737 non-null  object        
 5   Text           822737 non-null  object        
 6   TextWordCount  822737 non-null  object        
 7   PubTitle       822737 non-null  object        
 8   SourceType     822737 non-null  object        
 9   Year           822737 non-null  int64         
 10  Month          822737 non-null  int64         
 11  Newspaper      822737 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(9)
memory usage: 75.3+ MB
None


In [5]:
# print(allNews['PubTitle'].value_counts())
# print(allNews['PubTitle'].nunique())

In [6]:
print(allNews['Newspaper'].value_counts())

Wall Street Journal    251983
New York Times         125270
Los Angeles Times      121406
The Washington Post    116772
Chicago Tribune         90023
Boston Globe            78922
USA Today               38361
Name: Newspaper, dtype: int64


In [7]:
# Merge with metadata
df=df_regSentsExpand.merge(allNews[['ID','Title','StartDate','Newspaper','Year','Month','Type','SourceType']],
                           on='ID',how='left')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 822737 entries, 0 to 822736
Data columns (total 13 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   ID                           822737 non-null  object        
 1   RegSentsExpand               822737 non-null  object        
 2   NounChunksMatch              822737 non-null  int64         
 3   NounChunkMatchWords          822737 non-null  object        
 4   NounChunkMatchWordsFiltered  822737 non-null  object        
 5   NounChunkMatchFiltered       822737 non-null  int64         
 6   Title                        822737 non-null  object        
 7   StartDate                    822737 non-null  datetime64[ns]
 8   Newspaper                    822737 non-null  object        
 9   Year                         822737 non-null  int64         
 10  Month                        822737 non-null  int64         
 11  Type                      

In [8]:
# Remove duplicated articles
IDs_nodup=pd.read_csv('/home/ec2-user/SageMaker/New Uncertainty/IDs_no_duplicates.csv')
print(IDs_nodup.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 788516 entries, 0 to 788515
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   ID      788516 non-null  int64
dtypes: int64(1)
memory usage: 6.0 MB
None


In [9]:
df['ID']=df['ID'].astype('int64')
df=IDs_nodup.merge(df,on='ID',how='left').reset_index(drop=True)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 788516 entries, 0 to 788515
Data columns (total 13 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   ID                           788516 non-null  int64         
 1   RegSentsExpand               788516 non-null  object        
 2   NounChunksMatch              788516 non-null  int64         
 3   NounChunkMatchWords          788516 non-null  object        
 4   NounChunkMatchWordsFiltered  788516 non-null  object        
 5   NounChunkMatchFiltered       788516 non-null  int64         
 6   Title                        788516 non-null  object        
 7   StartDate                    788516 non-null  datetime64[ns]
 8   Newspaper                    788516 non-null  object        
 9   Year                         788516 non-null  int64         
 10  Month                        788516 non-null  int64         
 11  Type                      

In [10]:
# Reg relevant articles
print("# of reg relevent articles (unfiltered):",df[df['NounChunksMatch']!=0]['ID'].nunique())    # Unfiltered version of noun chunks

print("# of reg relevent articles (filtered):",df[df['NounChunkMatchFiltered']!=0]['ID'].nunique())    # Filtered version of noun chunks
#print(df['NounChunkMatchFiltered'].value_counts())

# of reg relevent articles (unfiltered): 553586
# of reg relevent articles (filtered): 493418


In [11]:
# Use filtered noun chunk matches to define reg relevance
df.loc[df['NounChunkMatchFiltered']!=0,'RegRelevance']=1
print(df.info())
print("# of reg relevant articles:",df[df['RegRelevance']==1]['ID'].nunique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 788516 entries, 0 to 788515
Data columns (total 14 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   ID                           788516 non-null  int64         
 1   RegSentsExpand               788516 non-null  object        
 2   NounChunksMatch              788516 non-null  int64         
 3   NounChunkMatchWords          788516 non-null  object        
 4   NounChunkMatchWordsFiltered  788516 non-null  object        
 5   NounChunkMatchFiltered       788516 non-null  int64         
 6   Title                        788516 non-null  object        
 7   StartDate                    788516 non-null  datetime64[ns]
 8   Newspaper                    788516 non-null  object        
 9   Year                         788516 non-null  int64         
 10  Month                        788516 non-null  int64         
 11  Type                      

In [13]:
# Aggregate reg-relevant article count by month
monthlyCount=df[['Newspaper','Year','Month','RegRelevance']].groupby(['Newspaper','Year','Month']).agg('sum').reset_index()
print(monthlyCount.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2945 entries, 0 to 2944
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Newspaper     2945 non-null   object 
 1   Year          2945 non-null   int64  
 2   Month         2945 non-null   int64  
 3   RegRelevance  2945 non-null   float64
dtypes: float64(1), int64(2), object(1)
memory usage: 92.2+ KB
None


In [14]:
print(monthlyCount)

                Newspaper  Year  Month  RegRelevance
0            Boston Globe  1985      1          83.0
1            Boston Globe  1985      2          91.0
2            Boston Globe  1985      3         107.0
3            Boston Globe  1985      4          93.0
4            Boston Globe  1985      5         113.0
...                   ...   ...    ...           ...
2940  Wall Street Journal  2020      4         380.0
2941  Wall Street Journal  2020      5         364.0
2942  Wall Street Journal  2020      6         381.0
2943  Wall Street Journal  2020      7         377.0
2944  Wall Street Journal  2020      8         369.0

[2945 rows x 4 columns]


In [15]:
monthlyCount.to_csv('/home/ec2-user/SageMaker/New Uncertainty/Reg Relevance/RegRelevant_MonthlyArticleCount.csv',index=False)